<a href="https://colab.research.google.com/github/guraseesrai/hr-analytics-employee-attrition/blob/main/SQL_Capstone_1_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Capstone 1 - Part 2

In the first part of the capstone, we focused on Data Retrieval, Data Preprocessing, Feature Engineering and Exploratory Data Analysis using Python & its libraries. Now we are going to shift gears and gain insights into our HR Analytics dataset using SQL.

## TODO: Make use of SQL to do the following:

### Create a SQLITE3 DB using the CSV file (2 pts). Please refer this [link](https://www.linkedin.com/pulse/accessing-sqlite3-database-from-jupyter-notebook-using-varun-lobo/) and this [link](https://www.geeksforgeeks.org/how-to-import-csv-file-in-sqlite-database-using-python/) to know more.

### Calculate the Attrition Rate and summarize attrition (3 pts) by:
- Gender
- Department
- Age
- Average monthly income by job level
- Years at company

### Continue using SQL to explore main reasons for attrition (3 pts), For example:
- Why do more people over 50 years old leave the company than people who aged 40-50?
- Why do people with higher pay still leave the company?
- Which factors drive employees who work at company less than 5 years to leave?

### Effective Communication (2 pts)
- Please make use of markdown cells to communicate your thought process, why did you think of performing a step? what was the observation from the query? etc.
- The code should be commented so that it is readable for the reviewer.

### Grading and Important Instructions
- Each of the above steps are mandatory and should be completed in good faith
- Make sure before submitting that the code is in fully working condition
- It is fine to make use of ChatGPT, stackoverflow type resources, just provide the reference links from where you got it
- Debugging is an art, if you find yourself stuck with errors, take help of stackoverflow and ChatGPT to resolve the issue and if it's still unresolved, reach out to me for help.
- You need to score atleast 7/10 to pass the project, anything less than that will be marked required, needing resubmission.
- Feedback will be provided on 3 levels (Awesome, Suggestion, & Required). Required changes are mandatory to be made.
- For submission, please upload the project on github and share the link to the file with us through LMS.

#### Calculate Attrition Rate


In [5]:
### Implement code
import pandas as pd
import sqlite3
from google.colab import drive

drive.mount('/content/drive')

# Load dataset
df = pd.read_csv('/content/drive/MyDrive/DA/HR-Analytics.csv')

# Create SQLite DB and table
conn = sqlite3.connect("hr_analytics.db")
df.to_sql("employees", conn, if_exists='replace', index=False)

# Test: Show first few rows
pd.read_sql("SELECT * FROM employees LIMIT 5", conn)


Mounted at /content/drive


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [6]:
# Overall attrition rate
query = """
WITH attrition_summary AS (
  SELECT
    CASE
      WHEN Attrition = 'Yes' THEN 'true'
      ELSE 'false'
    END AS Attrition,

    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM employees), 1) AS Attrition_Rate

  FROM employees
  GROUP BY Attrition
)
SELECT
  ROW_NUMBER() OVER (ORDER BY Attrition DESC) AS Row,
  Attrition,
  Attrition_Rate || '%' AS Attrition_Rate
FROM attrition_summary;
"""

pd.read_sql(query, conn).style.hide(axis='index')


Row,Attrition,Attrition_Rate
1,true,16.1%
2,false,83.9%


#### Find Attrition by Gender

In [7]:
### Implement code
# By Gender
query = """
WITH gender_attrition AS (
  SELECT
    CASE WHEN Attrition = 'Yes' THEN 'true' ELSE 'false' END AS Attrition,
    Gender,
    COUNT(*) AS Count_gender,
    ROUND(COUNT(*) * 100.0 / (
      SELECT COUNT(*) FROM employees e2 WHERE e2.Gender = e1.Gender
    ), 1) AS Attrition_by_gender
  FROM employees e1
  GROUP BY Gender, Attrition
)
SELECT
  ROW_NUMBER() OVER (ORDER BY Attrition DESC, Gender) AS Row,
  Attrition,
  Gender,
  Count_gender,
  Attrition_by_gender
FROM gender_attrition;
"""

import pandas as pd
df_result = pd.read_sql(query, conn).style.hide(axis='index')
df_result


Row,Attrition,Gender,Count_gender,Attrition_by_gender
1,true,Female,87,14.800000
2,true,Male,150,17.000000
3,false,Female,501,85.200000
4,false,Male,732,83.000000


#### Find Attrition by Dept

In [8]:
#### Find Attrition by Gender### Implement code

query = """
SELECT
  Department,
  Attrition,
  COUNT(*) AS Department_attrition
FROM employees
GROUP BY Department, Attrition
ORDER BY Department, Attrition DESC;
"""

pd.read_sql(query, conn).style.hide(axis='index')



Department,Attrition,Department_attrition
Human Resources,Yes,12
Human Resources,No,51
Research & Development,Yes,133
Research & Development,No,828
Sales,Yes,92
Sales,No,354


#### Find Attrition by Age Groups

In [13]:
query = """
WITH age_attrition AS (
  SELECT
    CASE
      WHEN Age < 30 THEN '<30'
      WHEN Age BETWEEN 30 AND 39 THEN '30-39'
      WHEN Age BETWEEN 40 AND 49 THEN '40-49'
      ELSE '50+'
    END AS Age_Group,

    CASE
      WHEN Attrition = 'Yes' THEN 'true'
      ELSE 'false'
    END AS Attrition,

    COUNT(*) AS Count_age_group,

    ROUND(COUNT(*) * 100.0 / (
      SELECT COUNT(*) FROM employees e2
      WHERE
        CASE
          WHEN e1.Age < 30 THEN '<30'
          WHEN e1.Age BETWEEN 30 AND 39 THEN '30-39'
          WHEN e1.Age BETWEEN 40 AND 49 THEN '40-49'
          ELSE '50+'
        END =
        CASE
          WHEN e2.Age < 30 THEN '<30'
          WHEN e2.Age BETWEEN 30 AND 39 THEN '30-39'
          WHEN e2.Age BETWEEN 40 AND 49 THEN '40-49'
          ELSE '50+'
        END
    ), 2) AS Attrition_percent
  FROM employees e1
  GROUP BY Age_Group, Attrition
)
SELECT
  ROW_NUMBER() OVER (ORDER BY Attrition, Attrition_percent DESC) AS Row,
  Age_Group,
  Attrition,
  Count_age_group,
  Attrition_percent
FROM age_attrition
ORDER BY Attrition, Attrition_percent DESC;
"""

pd.read_sql(query, conn).style.hide(axis='index')


Row,Age_Group,Attrition,Count_age_group,Attrition_percent
1,40-49,false,315,90.260000
2,50+,false,150,86.710000
3,30-39,false,533,85.690000
4,<30,false,235,72.090000
5,<30,true,91,27.910000
6,30-39,true,89,14.310000
7,50+,true,23,13.290000
8,40-49,true,34,9.740000


#### Find Attrition by Monthly Income

In [17]:
query = """
WITH income_by_group AS (
  SELECT
    Department,
    JobLevel,
    ROUND(AVG(MonthlyIncome), 1) AS avg_income
  FROM employees
  GROUP BY Department, JobLevel
),
attrition_income_by_group AS (
  SELECT
    Department,
    JobLevel,
    ROUND(AVG(MonthlyIncome), 1) AS attrition_avg_income
  FROM employees
  WHERE Attrition = 'Yes'
  GROUP BY Department, JobLevel
)
SELECT
  ROW_NUMBER() OVER (ORDER BY a.Department, a.JobLevel) AS Row,
  a.Department,
  a.JobLevel,
  printf("%.1f", a.avg_income) AS avg_income,
  printf("%.1f", b.attrition_avg_income) AS attrition_avg_income,
  printf("%.1f", b.attrition_avg_income - a.avg_income) AS difference
FROM income_by_group a
INNER JOIN attrition_income_by_group b
  ON a.Department = b.Department AND a.JobLevel = b.JobLevel;
"""

pd.read_sql(query, conn).style.hide(axis='index')


Row,Department,JobLevel,avg_income,attrition_avg_income,difference
1,Human Resources,1,2733.2,2415.7,-317.5
2,Human Resources,3,9623.0,10216.0,593.0
3,Research & Development,1,2840.1,2687.4,-152.7
4,Research & Development,2,5291.2,5372.0,80.8
5,Research & Development,3,10170.5,9503.8,-666.7
6,Research & Development,4,15634.7,12169.0,-3465.7
7,Research & Development,5,19218.5,19550.0,331.5
8,Sales,1,2506.7,2373.4,-133.3
9,Sales,2,5746.1,5917.0,170.9
10,Sales,3,9282.3,9202.8,-79.5


#### Find Attrition by Years At Company

In [18]:
### Implement code
query = """
WITH tenure_bins AS (
  SELECT
    CASE
      WHEN YearsAtCompany BETWEEN 0 AND 1 THEN 'New Hires'
      WHEN YearsAtCompany BETWEEN 2 AND 5 THEN '2-5 years'
      WHEN YearsAtCompany BETWEEN 6 AND 10 THEN '6-10 years'
      WHEN YearsAtCompany BETWEEN 11 AND 20 THEN '11-20 years'
      ELSE 'Over 20 years'
    END AS tenure_years,
    COUNT(*) AS num
  FROM employees
  WHERE Attrition = 'Yes'
  GROUP BY tenure_years
),
tenure_with_percent AS (
  SELECT
    tenure_years,
    num,
    ROUND(num * 100.0 / (SELECT COUNT(*) FROM employees WHERE Attrition = 'Yes'), 1) AS percent
  FROM tenure_bins
)
SELECT
  ROW_NUMBER() OVER (ORDER BY percent DESC) AS Row,
  tenure_years,
  num,
  printf("%.1f", percent) AS percent
FROM tenure_with_percent;
"""

pd.read_sql(query, conn).style.hide(axis='index')


Row,tenure_years,num,percent
1,2-5 years,87,36.7
2,New Hires,75,31.6
3,6-10 years,55,23.2
4,11-20 years,12,5.1
5,Over 20 years,8,3.4


#### Continue using SQL to explore main reasons for attrition (3 pts), For example:
- Why do more people over 50 years old leave the company than people who aged 40-50?
- Why do people with higher pay still leave the company?
- Which factors drive employees who work at company less than 5 years to leave?

In [21]:
query = """
/* Compare counts & rates for 40-50 vs Over 50 */
WITH age_buckets AS (
  SELECT
         CASE WHEN Age BETWEEN 40 AND 50 THEN '40-50'
              WHEN Age > 50              THEN 'Over 50'
         END                           AS age_grp,
         COUNT(*)                      AS total,
         SUM(CASE WHEN Attrition='Yes' THEN 1 ELSE 0 END) AS left_cnt
  FROM   employees
  WHERE  Age >= 40
  GROUP  BY age_grp
)
SELECT
  age_grp,
  left_cnt,
  total,
  ROUND( left_cnt*100.0 / total , 1 ) AS attrition_rate
FROM   age_buckets
ORDER  BY attrition_rate DESC;
"""

pd.read_sql(query, conn).style.hide(axis='index')


age_grp,left_cnt,total,attrition_rate
Over 50,18,143,12.600000
40-50,39,379,10.300000


In [29]:
query = """
/* 1️⃣  collect leavers 40+ */
WITH leavers AS (
  SELECT
         CASE WHEN Age BETWEEN 40 AND 50 THEN '40-50'
              ELSE 'Over 50' END          AS age_grp,
         YearsSinceLastPromotion          AS yslp
  FROM   employees
  WHERE  Attrition = 'Yes'
    AND  Age >= 40
),

/* 2️⃣  add row numbers inside each age bucket */
ordered AS (
  SELECT
         l.*,
         ROW_NUMBER() OVER (PARTITION BY age_grp ORDER BY yslp)             AS rn,
         COUNT(*)     OVER (PARTITION BY age_grp)                           AS n
  FROM   leavers l
),

/* 3️⃣  pick the median row(s) */
median_calc AS (
  SELECT age_grp,
         CASE                               -- handles odd / even counts
           WHEN n % 2 = 1 AND rn = (n+1)/2       THEN 1
           WHEN n % 2 = 0 AND rn IN (n/2, n/2+1) THEN 1
           ELSE 0
         END AS is_median,
         yslp
  FROM   ordered
)

SELECT
  age_grp,
  ROUND(AVG(yslp),1)                                                   AS avg_yslp,
  ROUND(AVG(CASE WHEN is_median=1 THEN yslp END),1)                    AS median_yslp
FROM   median_calc
GROUP BY age_grp
ORDER BY age_grp;

"""
pd.read_sql(query, conn).style.hide(axis='index')



age_grp,avg_yslp,median_yslp
40-50,2.600000,1.000000
Over 50,4.600000,3.500000


In [30]:
query = """
WITH leavers AS (
  SELECT Age, WorkLifeBalance
  FROM   employees
  WHERE  Attrition='Yes' AND Age >= 40
),
bucketed AS (
  SELECT
    CASE WHEN Age BETWEEN 40 AND 50 THEN '40-50'
         ELSE 'Over 50' END AS age_grp,
    WorkLifeBalance
  FROM   leavers
),
counts AS (
  SELECT age_grp,
         WorkLifeBalance,
         COUNT(*) AS n
  FROM   bucketed
  GROUP  BY age_grp, WorkLifeBalance
),
tot AS (
  SELECT age_grp, SUM(n) AS tot
  FROM   counts
  GROUP  BY age_grp
)
SELECT
  c.age_grp,
  c.WorkLifeBalance AS WLB_score,
  ROUND(c.n*100.0 / t.tot , 1) AS pct_of_leavers
FROM   counts c
JOIN   tot t USING(age_grp)
WHERE  c.WorkLifeBalance = 1
ORDER  BY c.age_grp;
"""

pd.read_sql(query, conn).style.hide(axis='index')


age_grp,WLB_score,pct_of_leavers
40-50,1,15.400000
Over 50,1,5.600000


### 📊 Why do employees **over 50** leave more than those aged 40-50?

| Age Group | Leavers ( `left_cnt` ) | Headcount ( `total` ) | Attrition Rate |
|-----------|-----------------------|-----------------------|----------------|
| **Over 50** | **18** | **143** | **12.6 %** |
| 40-50      | 39      | 379     | 10.3 % |

| Age Group | Avg Years Since Last Promotion | Median Years Since Last Promotion |
|-----------|--------------------------------|-----------------------------------|
| **Over 50** | **4.6 yrs** | **3.5** |
| 40-50      | 2.6 yrs | 1.0 |

| Age Group | % of Leavers with **Work-Life Balance = 1** (worst) |
|-----------|----------------------------------------------------|
| 40-50      | 15.4 % |
| **Over 50** | **5.6 %** |

**Interpretation**

* Over-50 staff leave at a **higher risk**—**12.6 % vs 10.3 %**, a ~22 % relative increase.  
* They have waited **~2 extra years** since their last promotion, pointing to **career stagnation / plateau**.  
* Only **5.6 %** of over-50 leavers rate WLB at the worst level (vs 15 % in the 40-50 group), so work-life strain is *not* the driver here.  

**Conclusion → Late-career drivers**

> The elevated attrition among employees over 50 is linked to a **longer promotion drought** and likely retirement planning, rather than burnout.  
> Recommended actions: introduce phased-retirement pathways and targeted late-career development plans to retain critical expertise.


In [31]:
query = """
/* Quartile cut-offs → Q4 = top 25 % earners */
WITH q AS (
  SELECT MonthlyIncome,
         NTILE(4) OVER (ORDER BY MonthlyIncome) AS inc_q
  FROM   employees
),
attr AS (
  SELECT inc_q,
         COUNT(*)                                    AS total,
         SUM(CASE WHEN Attrition='Yes' THEN 1 ELSE 0 END) AS left_cnt
  FROM   employees e
  JOIN   q ON e.MonthlyIncome = q.MonthlyIncome
  GROUP  BY inc_q
)
SELECT
  inc_q                AS income_quartile,   -- 4 = highest pay
  left_cnt,
  total,
  ROUND(left_cnt*100.0/total,1) AS attrition_rate
FROM   attr
ORDER  BY inc_q DESC;        -- shows Q4 (highest earners) first


"""

pd.read_sql(query, conn).style.hide(axis='index')


income_quartile,left_cnt,total,attrition_rate
4,41,379,10.800000
3,46,437,10.500000
2,58,429,13.500000
1,145,493,29.400000


In [32]:
query = """/* Pull key workplace factors for Q4 leavers */
SELECT OverTime,
       WorkLifeBalance,
       JobRole,
       COUNT(*) AS n
FROM   employees
WHERE  Attrition='Yes'
  AND  MonthlyIncome IN (   -- Q4 filter
        SELECT MonthlyIncome
        FROM   (SELECT MonthlyIncome,
                       NTILE(4) OVER (ORDER BY MonthlyIncome) AS q
                FROM   employees)
        WHERE  q = 4 )
GROUP  BY OverTime, WorkLifeBalance, JobRole
ORDER  BY n DESC
LIMIT 15;"""
pd.read_sql(query, conn).style.hide(axis='index')

OverTime,WorkLifeBalance,JobRole,n
Yes,2,Sales Executive,7
Yes,3,Sales Executive,5
No,3,Sales Executive,4
No,3,Manufacturing Director,3
No,1,Sales Executive,2
No,2,Healthcare Representative,2
No,3,Healthcare Representative,2
Yes,3,Manager,2
No,2,Sales Executive,1
No,3,Human Resources,1


###  Why do some high-paid employees still leave?

#### (1) Attrition by Income Quartile  
| Income Quartile | Leavers (`left_cnt`) | Workforce Size | Attrition Rate |
|-----------------|----------------------|----------------|----------------|
| **Q4 (top 25 %)** | **41** | 379 | **10.8 %** |
| Q3 (50-75 %)     | 46  | 437 | 10.5 % |
| Q2 (25-50 %)     | 58  | 429 | 13.5 % |
| **Q1 (bottom 25 %)** | **145** | 493 | **29.4 %** |

*High pay lowers risk, but **41 very well-paid employees still quit**.*

#### (2) Who are those 41 Q4 leavers?  
| OverTime | Work-Life Balance | Job Role                | n |
|----------|------------------|-------------------------|---|
| **Yes**  | 2 | **Sales Executive**         | **7** |
| **Yes**  | 3 | Sales Executive            | 5 |
| No       | 3 | Sales Executive            | 4 |
| No       | 3 | Manufacturing Director     | 3 |
| No       | 1 | Sales Executive            | 2 |
| No       | 2 | Healthcare Representative  | 2 |
| No       | 3 | Healthcare Representative  | 2 |
| Yes      | 3 | Manager                    | 2 |
| *(single-occurrence roles omitted for brevity; see full table above)* |

####  Interpretation  
* **Role-specific stress.** Over half of Q4 leavers are **Sales Executives**; others hold high-pressure director or manager titles.  
* **Chronic overtime.** > 50 % of Q4 leavers worked regular overtime.  
* **Work-life fatigue.** Nearly all report Work-Life Balance scores of 2–3 (mid-to-poor); a few score 1 (worst).  
* **Conclusion:** Even at top pay, employees exit when **workload and job stress outweigh compensation**—especially in demanding client-facing or operational leadership roles.

#### Recommendations  
1. **Cap overtime** for senior sales & manufacturing staff; rotate coverage.  
2. Introduce **executive flex schedules** and wellness stipends.  
3. Create late-career growth tracks (e.g., Principal Sales Exec, Plant Fellow) to avoid plateau.

> **Bottom line:** Salary retains talent only up to the point where **overwork and limited growth** tip the scales—the data show exactly where that happens.


In [24]:

query = """
SELECT OverTime,
       COUNT(*)                                    AS num,
       ROUND(COUNT(*)*100.0/
             (SELECT COUNT(*) FROM employees WHERE YearsAtCompany<5 AND Attrition='Yes'),1) AS percent
FROM   employees
WHERE  Attrition='Yes'
  AND  YearsAtCompany < 5
GROUP  BY OverTime
ORDER  BY percent DESC;
"""

pd.read_sql(query, conn).style.hide(axis='index')


OverTime,num,percent
Yes,75,53.200000
No,66,46.800000


In [25]:

query = """
SELECT WorkLifeBalance,
       COUNT(*) AS num,
       ROUND(COUNT(*)*100.0/
             (SELECT COUNT(*) FROM employees WHERE YearsAtCompany<5 AND Attrition='Yes'),1) AS percent
FROM   employees
WHERE  Attrition='Yes'
  AND  YearsAtCompany < 5
GROUP  BY WorkLifeBalance
ORDER  BY percent DESC;

"""

pd.read_sql(query, conn).style.hide(axis='index')


WorkLifeBalance,num,percent
3,76,53.900000
2,35,24.800000
4,16,11.300000
1,14,9.900000


In [26]:
query = """
SELECT JobRole,
       COUNT(*) AS num_leavers
FROM   employees
WHERE  Attrition='Yes'
  AND  YearsAtCompany < 5
GROUP  BY JobRole
ORDER  BY num_leavers DESC
LIMIT 10;


"""

pd.read_sql(query, conn).style.hide(axis='index')

JobRole,num_leavers
Laboratory Technician,46
Sales Representative,30
Research Scientist,26
Sales Executive,23
Human Resources,9
Manufacturing Director,4
Healthcare Representative,3


###  What drives < 5-year employees to leave?

| Factor                               | Evidence from SQL output | Insight |
|--------------------------------------|--------------------------|---------|
| **Over-time workload**               | 53 % of early leavers worked **OverTime = Yes** (75 of 141) | Heavy hours are the single strongest early-tenure trigger. |
| **Work-Life Balance**                | A majority of < 5-yr leavers rate WLB = 3 (54 %) or 2 (25 %); only 10 % rate it 1 (worst). | They are *not* at the very worst WLB, but “fair-to-poor” (2–3) dominates—suggesting workday pressure that pay alone can’t offset. |
| **Role type**                        | Top exit roles: **Laboratory Technician (46)**, **Sales Representative (30)**, **Research Scientist (26)**, **Sales Executive (23)**. | Hands-on lab work and quota-bearing sales jobs churn fastest in the first five years. |

####  Interpretation

* **Workload > Pay** – Over-time is present in **half of all early exits**, hinting at burnout before loyalty forms.  
* **“Middle” Work-Life Balance isn’t good enough** – Most new staff who leave give a middling WLB score (2–3). They’re *not* suffering extreme conditions, yet they still exit—showing early expectations aren’t met.  
* **Job-Role effect** – Lab techs and junior sales scientists exit earliest, possibly due to repetitive tasks or quota pressure.

####  Recommendations

1. **Cap overtime for employees in their first 3 years**; monitor via timesheets.  
2. **Early-career WLB check-ins** at 6- and 18-month marks; fast-track fixes if scores fall to 2–3.  
3. **Mentorship & skill-rotation programs** for Laboratory Techs and Sales Reps to keep work varied and career paths clear.

> **Bottom line:** Early-tenure attrition is driven less by pay and more by **workload intensity and role design**—tackle those and you keep new talent.
